In [1]:
import os
import pandas as pd
from statsmodels.tsa.api import VAR
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import numpy as np
import pickle

In [2]:
# Folder paths
input_folder = 'city_wise_data_splits'  # City CSVs named like Colombo_train.csv, Colombo_val.csv, Colombo_test.csv
model_folder = 'trained_var_models'
os.makedirs(model_folder, exist_ok=True)

In [3]:
# Cities list (modify if needed)
cities = [
    "Colombo", "Gampaha", "Kalutara", "Kandy", "Matale", "Nuwara_Eliya", "Galle", "Matara",
    "Hambantota", "Jaffna", "Kilinochchi1", "Mannar", "Vavuniya", "Mullaitivu", "Batticaloa",
    "Ampara", "Trincomalee", "Kurunegala", "Puttalam", "Anuradhapura", "Polonnaruwa", "Badulla",
    "Moneragala", "Ratnapura", "Kegalle", "Welimada", "Bandarawela"
]

In [4]:
def evaluate_model(true_df, pred_df):
    print("Evaluation metrics:")
    for col in true_df.columns:
        y_true = true_df[col].values
        y_pred = pred_df[col].values
        print(f"  {col}:")
        print(f"    MAE  = {mean_absolute_error(y_true, y_pred):.4f}")
        print(f"    RMSE = {np.sqrt(mean_squared_error(y_true, y_pred)):.4f}")
        print(f"    R2   = {r2_score(y_true, y_pred):.4f}")

In [5]:
for city in cities:
    city_safe = city.replace(' ', '_').replace('[', '').replace(']', '')
    try:
        # Load train, val, test CSVs for this city
        train_path = os.path.join(input_folder, f"{city_safe}_train.csv")
        val_path = os.path.join(input_folder, f"{city_safe}_val.csv")
        test_path = os.path.join(input_folder, f"{city_safe}_test.csv")

        # Check if required files exist
        if not (os.path.isfile(train_path) and os.path.isfile(val_path) and os.path.isfile(test_path)):
            print(f"[SKIP] Missing train/val/test files for city: {city}")
            continue

        # Load data
        train_df = pd.read_csv(train_path, parse_dates=['date']).sort_values('date').set_index('date')
        val_df = pd.read_csv(val_path, parse_dates=['date']).sort_values('date').set_index('date')
        test_df = pd.read_csv(test_path, parse_dates=['date']).sort_values('date').set_index('date')

        # Required columns check
        required_cols = ['temperature', 'rainfall', 'wind_speed']
        if not all(col in train_df.columns for col in required_cols):
            print(f"[SKIP] Train CSV missing required columns for city: {city}")
            continue

        # Select only required columns
        train_data = train_df[required_cols]
        val_data = val_df[required_cols]
        test_data = test_df[required_cols]

        # Train VAR model
        model = VAR(train_data)
        lag_order_results = model.select_order(maxlags=15)
        lag_order = lag_order_results.aic
        print(f"\nTraining VAR model for city: {city}")
        print(f"Selected lag order (AIC): {lag_order}")

        results = model.fit(lag_order)

        # Validate
        forecast_vals = results.forecast(train_data.values[-lag_order:], steps=len(val_data))
        forecast_df = pd.DataFrame(forecast_vals, index=val_data.index, columns=val_data.columns)

        print(f"Validation results for city: {city}")
        evaluate_model(val_data, forecast_df)

        # Save the trained model as pickle file
        model_filename = os.path.join(model_folder, f"{city_safe}_var_model.pkl")
        with open(model_filename, 'wb') as f:
            pickle.dump(results, f)
        print(f"Saved VAR model for {city} at {model_filename}")

    except Exception as e:
        print(f"[ERROR] Failed processing city {city}: {e}")


c:\Users\HP\AppData\Local\Programs\Python\Python313\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency D will be used.
  self._init_dates(dates, freq)



Training VAR model for city: Colombo
Selected lag order (AIC): 12
Validation results for city: Colombo
Evaluation metrics:
  temperature:
    MAE  = 0.7010
    RMSE = 0.8694
    R2   = 0.0253
  rainfall:
    MAE  = 6.6587
    RMSE = 10.4273
    R2   = 0.0049
  wind_speed:
    MAE  = 3.6303
    RMSE = 4.6319
    R2   = -0.1807
Saved VAR model for Colombo at trained_var_models\Colombo_var_model.pkl

Training VAR model for city: Gampaha
Selected lag order (AIC): 15


c:\Users\HP\AppData\Local\Programs\Python\Python313\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency D will be used.
  self._init_dates(dates, freq)
c:\Users\HP\AppData\Local\Programs\Python\Python313\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency D will be used.
  self._init_dates(dates, freq)


Validation results for city: Gampaha
Evaluation metrics:
  temperature:
    MAE  = 0.6947
    RMSE = 0.8695
    R2   = 0.1235
  rainfall:
    MAE  = 6.7506
    RMSE = 10.1815
    R2   = 0.0047
  wind_speed:
    MAE  = 2.9539
    RMSE = 3.6597
    R2   = -0.0189
Saved VAR model for Gampaha at trained_var_models\Gampaha_var_model.pkl

Training VAR model for city: Kalutara
Selected lag order (AIC): 15
Validation results for city: Kalutara
Evaluation metrics:
  temperature:
    MAE  = 0.6976
    RMSE = 0.8481
    R2   = -0.0197
  rainfall:
    MAE  = 6.2365
    RMSE = 8.6355
    R2   = 0.0017
  wind_speed:
    MAE  = 3.3297
    RMSE = 4.3035
    R2   = -0.0398
Saved VAR model for Kalutara at trained_var_models\Kalutara_var_model.pkl


c:\Users\HP\AppData\Local\Programs\Python\Python313\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency D will be used.
  self._init_dates(dates, freq)


...

c:\Users\HP\AppData\Local\Programs\Python\Python313\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency D will be used.
  self._init_dates(dates, freq)



Training VAR model for city: Kegalle
Selected lag order (AIC): 11
Validation results for city: Kegalle
Evaluation metrics:
  temperature:
    MAE  = 0.8567
    RMSE = 1.0897
    R2   = 0.1270
  rainfall:
    MAE  = 6.7086
    RMSE = 10.4154
    R2   = -0.0033
  wind_speed:
    MAE  = 3.0564
    RMSE = 3.7712
    R2   = -0.0684
Saved VAR model for Kegalle at trained_var_models\Kegalle_var_model.pkl


c:\Users\HP\AppData\Local\Programs\Python\Python313\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency D will be used.
  self._init_dates(dates, freq)



Training VAR model for city: Welimada
Selected lag order (AIC): 11
Validation results for city: Welimada
Evaluation metrics:
  temperature:
    MAE  = 1.2548
    RMSE = 1.5623
    R2   = -0.4132
  rainfall:
    MAE  = 6.0776
    RMSE = 9.3048
    R2   = -0.0373
  wind_speed:
    MAE  = 2.7674
    RMSE = 3.6372
    R2   = -0.0125
Saved VAR model for Welimada at trained_var_models\Welimada_var_model.pkl

Training VAR model for city: Bandarawela
Selected lag order (AIC): 10
Validation results for city: Bandarawela
Evaluation metrics:
  temperature:
    MAE  = 1.1917
    RMSE = 1.4661
    R2   = -0.5732
  rainfall:
    MAE  = 6.0720
    RMSE = 9.6158
    R2   = -0.0215
  wind_speed:
    MAE  = 2.6813
    RMSE = 3.3794
    R2   = 0.0053
Saved VAR model for Bandarawela at trained_var_models\Bandarawela_var_model.pkl


c:\Users\HP\AppData\Local\Programs\Python\Python313\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency D will be used.
  self._init_dates(dates, freq)
